In [4]:
import requests
import json
import hashlib
import os
from datetime import datetime as datingdays
from git import Repo, Git
import sys
project_root_path = '../../..'
python_lib_path = project_root_path + '/python/lib'
sys.path.append(python_lib_path)
from commit_log_parser import StatRequirementSet
from pytz import timezone

def introspect(obj):
  for func in [type, id, dir, vars, callable]:
        print("%s(%s):\t\t%s" % (func.__name__, introspect.__code__.co_varnames[0], func(obj)))
        
def loadCachedURL(url, forceReload = False):
    body = None
    hl = hashlib.new('sha256')
    ba = bytearray(url.encode())
    hl.update(ba)
    thing = hl.hexdigest()
    cachedFileName = './'+thing
    loaded = False;
    if not forceReload:
        try:
            with open(cachedFileName, 'r') as f:
                body = json.load(f)
                f.close()
            loaded = True
        except EnvironmentError:
            pass

    if not loaded:
        resp = requests.get(url)
        if (resp.status_code == 200):
            body = resp.json()
            with open(cachedFileName, 'w') as f:
                f.write(resp.text)
                f.close()
    return body
class Commit:
    sha = None
    date = None
    hacker = None
class RepoName:
    def __init__(self, owner, repo_name):
        self.owner = owner
        self.repo_name = repo_name
    def __hash__(self):
        return hash((self.owner, self.repo_name))

    def __eq__(self, other):
        return (self.owner, self.repo_name) == (other.owner, other.repo_name)

    def __ne__(self, other):
        return not(self == other)        
class Hacker:
    def __init__(self):
        self.user_id = None
        self.commits = []
        self.aliases = Counter()
    def __init__(self, user_id):
        self.user_id = user_id;
class Repository:
    def __init__(self, repo_id, repo_name):
        self.repo_id = repo_id
        self.repo_name = repo_name
class MyCounter:
    def __init__(self, init_val):
        self.counter = init_val
    def __init__(self):
        self.__init__(0)
    def increment(self):
        self.count += 1
    def val(self):
        return self.counter
    
# Types of queries:
#
#  Single query to derive a user ID
#  Mass commit log
class Query:
    def __init__(self):
        self.urlPrefix = 'https://api.github.com/search/commits'
        self.startDate = datingdays.now(timezone('US/Arizona'))
        self.hackers = {}
        self.repos = {}
        self.aliases = {}
        self.resolved_alias_map = {}
        self.commit_to_repo_map = {}
        with open('./web3.github.token', 'r') as f:
            self.token = f.readline()
            self.token = self.token.strip('\n')
            self.headers = {'Authorization': 'token %s' % self.token}
    def add_alias(self, alias, commit_key):
        if (alias not in self.aliases.keys()):
            self.aliases[alias] = []
        self.aliases[alias].append(commit_key)
    
    def set_last_date(self, date):
        new_date = datingdays.fromisoformat(date)
        if (new_date < self.startDate):
#           Note: GitHub rejected dates with timezones other than "-07:00" (like "+02:00")
#                 By subtracting the difference (in milliseconds?) we represent the "US/Arizona"
#                 version of the author-date pulled from previous results
            self.startDate = self.startDate - (self.startDate - new_date)
    def format_url(self):
        var = self.urlPrefix+"?q=author:"+self.hacker.user_id+'+author-date:<'+self.startDate.isoformat()+'&sort=author-date&order=desc&per_page=100&page=1'
        print(var)
        return var
    def add_commit_id(self, commit_id, repo_name):
        self.commit_to_repo_map[commit_id] = repo_name
    def format_id_check_url(self, commit_id):
        rn = self.commit_to_repo_map[commit_id]
        return 'https://api.github.com/repos/'+rn.owner+'/'+rn.repo_name+'/commits/'+commit_id
    def resolve_aliases(self):
        for a in self.aliases.keys():
            commit_id = q.aliases[a][0]  #Lookup just the first one
            print('Resolving ['+a+'] using commit ID: '+commit_id)
            url = self.format_id_check_url(commit_id)
            resp = requests.get(self.format_id_check_url(commit_id), headers=self.headers)
            if (resp.status_code == 200):
                j = resp.json()
                commit_details_block = j['author']
                if (commit_details_block == None):
                    commit_details_block = j['committer']
                if (commit_details_block == None):
                    print('Unable to find author node within JSON formatted result set')
                    print(resp)
                    print(json.dumps(j))
                else:
                    committer = commit_details_block['login']
                    if (committer not in self.hackers.keys()):
                        print('Creating new hacker object for '+committer+' ['+a+']')
                        self.hackers[committer] = []
                    else:
                        n = len(self.hackers[committer]) + 1
                        print('Adding alias ['+a+'] for user '+committer+' for a total of ', n)
                    self.hackers[committer].append(q.aliases[alias])
                    self.resolved_alias_map[a] = committer
                    
            
    def load_url(self):
        retVal = None
        resp = requests.get(self.format_url(), headers=self.headers)
        if (resp.status_code == 200):
            retVal = resp.json()
        return retVal
    
with open('./repos.json', "r") as r:
    array = json.load(r)
q = Query()

for n in array:
    owner = n['owner']
    repo_name = n['repo']
    repo = RepoName(owner, repo_name)
    repo_base_dir = './repos'
    repo_path = repo_base_dir+'/'+repo.owner+"/"+repo.repo_name

    if (os.path.isdir(repo_base_dir) == False):
        print('######### Cannot find '+repo_base_dir+'  Creating it!')
        os.makedirs(repo_base_dir)
    if (os.path.isdir(repo_base_dir+"/"+owner) == False):
        os.makedirs(repo_base_dir+"/"+owner)
    url = 'https://github.com/'+owner+'/'+repo_name+'.git'
    if (os.path.isdir(repo_path) == False):
        Repo.clone_from(url, repo_path)
    else:
        rp = Repo(repo_path)
        remote = rp.remote()
        remote.pull()
# Add call to rep.log('-1') to get the date from the latest change
#  If that date is less than the date on the cached stats file
#  then skip this one by loading the previous stats file.
    print('Generating Stats for '+repo_path)
    rep = Git(repo_path)
    stat = rep.log('--stat')

    stat_req_set = StatRequirementSet()
    stat_req_set.processDocument(stat)
    q.repos[repo] = stat_req_set.resultArray.copy()
    for rae in stat_req_set.resultArray:
        commit_id = rae['commit']
        alias = rae['Author']
        q.add_alias(alias, commit_id)
        q.add_commit_id(commit_id, repo)
        
print('Done loading!')
q.resolve_aliases()

with open('./hackers.json', 'w') as out:
    out.write(json.dumps(q.hackers))
with open('./aliasMap.json', 'w') as out:
    out.write(json.dumps(q.resolved_alias_map))
    
with open('./new_repos.json', 'w') as out:
    out.write(json.dumps(q.repos, indent=2))
for alias in q.aliases.keys():
    v = q.aliases[alias]
    print('Alias ', alias, ' has ', len(v), ' commits')
    
q.hacker = Hacker('marhali')    
done = True
while not done:
    body = q.load_url()
     
    if (body == None):
        print('Unable to load JSON')
        done = True
    else:
        total_count = body['total_count']
        incomplete_results = body['incomplete_results']
        print(total_count)
        print(incomplete_results)
        array = body['items'];
        if (array == None or len(array) < 1):
            done = True
        else:
            for n in array:
                author = n['author']
                login = author['login']
                repo = n['repository']
                repoId = repo['id']
                repoName = repo['name']
                commit = n['commit']
                sha = n['sha']
                comAuth = commit['author']
                authorName = comAuth['name']
                authorDate = comAuth['date']
                q.set_last_date(authorDate)
                print(login, repoId, repoName, authorName, authorDate, sha)
            if (total_count < 100 and incomplete_results == False):
                done = True
        


Generating Stats for ./repos/Deadman-DAO/Web3HackerNetwork
Generating Stats for ./repos/enigmatt/new2Web3
Generating Stats for ./repos/oceanprotocol/aquarius
Generating Stats for ./repos/oceanprotocol/contracts
Generating Stats for ./repos/oceanprotocol/market
Generating Stats for ./repos/oceanprotocol/ocean.js
Generating Stats for ./repos/oceanprotocol/ocean.py
Generating Stats for ./repos/oceanprotocol/provider
Generating Stats for ./repos/pypa/warehouse
Done loading!
Resolving [DeadmanBob <github@traxel.com>] using commit ID: 59be56523445605cf65d6439d94db9b23f45b3f4
Creating new hacker object for rbb36 [DeadmanBob <github@traxel.com>]
Resolving [enigmatt <git@theenkes.com>] using commit ID: 74901305894596853ac44b4d100c2d6c98a01959
Creating new hacker object for enigmatt [enigmatt <git@theenkes.com>]
Resolving [enigmatt <commerce@theenkes.com>] using commit ID: ddec3fefcf20ebe8e5427a615b50abbea89f8b39
Adding alias [enigmatt <commerce@theenkes.com>] for user enigmatt for a total of  2

TypeError: keys must be str, int, float, bool or None, not RepoName

In [ ]:
from datetime import datetime as datingdays
fmt = '%Y-%m-%dT%H:%M:%S.%f%z'
t = '2020-01-28T15:47:53.000+01:00'
rslt = datingdays.strptime(t, fmt)
print(rslt)
print(rslt.strftime(fmt))


In [ ]:
from datetime import datetime as dt
from pytz import timezone
d = dt.fromisoformat('2022-04-11T19:14:33.000+02:00')

#az = timezone('US/Arizona')
#d2 = az.localize(d, is_dst=False)
#d2              


In [ ]:
from datetime import datetime
from datetime import timedelta
# Given timestamp in string
time_str = '24/7/2021 11:13:08.230010'
date_format_str = '%d/%m/%Y %H:%M:%S.%f'
# create datetime object from timestamp string
given_time = datetime.strptime(time_str, date_format_str)
print('Given Time: ', given_time)
n = 2
# Subtract 2 hours from datetime object
final_time = given_time - timedelta(hours=n)
print('Final Time (2 hours ahead of given time ): ', final_time)
# Convert datetime object to string in specific format 
final_time_str = final_time.strftime('%d/%m/%Y %H:%M:%S.%f')
print('Final Time as string object: ', final_time_str)

In [ ]:
url = 'https://api.github.com/search/commit/fb5f372203f70cc7580f8e9806c00405524649d7